In [135]:
import os
import pandas as pd

In [136]:
df = pd.read_excel("/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels/Labelling_Prateek_Guillermo.xlsx")

In [137]:
def code_preprocessing(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
    comment_lines = []
    raw_codeLines = codeLines.replace("\t","").split("\n")
    multi_line_flag = 0
    for line_number in range(len(raw_codeLines)):
        if multi_line_flag:
            if raw_codeLines[line_number].__contains__("*/"):
                multi_line_flag = 0
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("/*"):
            if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                comment_lines.append(line_number)
                multi_line_flag = 1
            elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                if raw_codeLines[line_number].__contains__("*/"):
                    psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                    psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                    temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                    raw_codeLines[line_number] = temporary_line

        elif raw_codeLines[line_number].startswith("//"):
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("//"):
            comment_start = raw_codeLines[line_number].find('//')
            raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
    
    

    comment_lines.reverse()
    for i in comment_lines:
        raw_codeLines.pop(i)

    def insert_space(string, index):
        string_copy = ""
        for i in range(len(string)):
            if i==(index):
                string_copy += " "
                string_copy += string[i]
                string_copy += " "
                continue
            string_copy += string[i]
        return string_copy

    def find_char_indices(input_string, char):
        indices = []
        replacement_token = 0
        for index, character in enumerate(input_string):
            if character == char:
                indices.append(index + 2*replacement_token)
                replacement_token+=1
        return indices

    def space_out(string, char):
        indices = find_char_indices(string, char)

        for i in indices:
            string = insert_space(string, i)
        return string

    for line_number in range(len(raw_codeLines)):
        placeHolder = raw_codeLines[line_number]
        placeHolder = space_out(placeHolder, ";")
        placeHolder = space_out(placeHolder, "(")
        placeHolder = space_out(placeHolder, ")")
        placeHolder = space_out(placeHolder, ",")
        if placeHolder.endswith(";"):
            raw_codeLines[line_number] = "<start> " + placeHolder.replace(";","<end>")
        elif not(placeHolder.endswith(";")):
            raw_codeLines[line_number] = "<start> " + placeHolder + " <end>"

    return raw_codeLines


In [138]:
def comment_finder(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
        comment_lines = []
        raw_codeLines = codeLines.replace("\t","").split("\n")
        multi_line_flag = 0
        for line_number in range(len(raw_codeLines)):
            if multi_line_flag:
                if raw_codeLines[line_number].__contains__("*/"):
                    multi_line_flag = 0
                comment_lines.append(line_number)
            elif raw_codeLines[line_number].__contains__("/*"):
                if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                    comment_lines.append(line_number)
                    multi_line_flag = 1
                elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                    if raw_codeLines[line_number].__contains__("*/"):
                        psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                        psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                        temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                        raw_codeLines[line_number] = temporary_line

            elif raw_codeLines[line_number].startswith("//"):
                comment_lines.append(line_number)
            elif raw_codeLines[line_number].__contains__("//"):
                comment_start = raw_codeLines[line_number].find('//')
                raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
    return comment_lines

In [139]:
file_start = {}
for i in range(len(df['File'])):
    if pd.isnull(df['File'][i]) == False:
        file_start[df['File'][i]] = i

BOTTOM_LINE = file_start['TOTAL NO. OF FILES']
file_list = list(file_start.keys())[:-1]
file_vulnerabilities = {}

for i in range(len(file_list)-1):
    vulnerable_lines=[]
    for j in range(file_start[file_list[i]],file_start[file_list[i+1]]-1):
        if (df['Comment'][j].split('|')[1].split()[0]) == "BRANCH:":
            if (pd.isna(df.at[j,'False Positive'])):
                vulnerable_lines.append(int(df['Comment'][j].split('|')[0].split()[1]))
        if (not(pd.isna(df.at[j,'Lines Missed']))):
            if (not(pd.isna(df.at[j,'Branch.1']))):
                line = df['Lines Missed'][j].split()
                if (not(line[0].isnumeric())):
                    vulnerable_lines.append(int(line[1].strip(":")))
                else:
                    vulnerable_lines.append(int(line[0].strip(":")))
    vulnerable_lines.sort()

    file_vulnerabilities[file_list[i]]=vulnerable_lines

file_list = file_list[:-1]

In [140]:
os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/dataset")
for file in file_list:
    comments = comment_finder(file)
    for vul_line in range(len(file_vulnerabilities[file])):
        for num in range(len(comments)):
            if num == len(comments)-1:
                (file_vulnerabilities[file][vul_line]) = ((file_vulnerabilities[file][vul_line])-(len(comments)))
            elif comments[num] < (file_vulnerabilities[file][vul_line]):
                None
            elif comments[num] > (file_vulnerabilities[file][vul_line]):
                (file_vulnerabilities[file][vul_line]) = ((file_vulnerabilities[file][vul_line])-(num+1))
                break


In [141]:
file_vulnerabilities[file]

[58, 112, 124, 187]

In [144]:
code = code_preprocessing(file)
len(code)

237

In [145]:
labeled_dataset = {'Lines':[], 'Label':[]}
labeled_dataset = pd.DataFrame(labeled_dataset)
labeled_dataset

for file in file_list:
    code = code_preprocessing(file)
    for line in range(len(code)):
        if (line) in file_vulnerabilities[file]:
            data = {'Lines': code[line], 'Label': 'Insecure'}
        else:
            data = {'Lines': code[line], 'Label': 'Secure'}
        labeled_dataset.loc[len(labeled_dataset)] = data

In [153]:
labeled_dataset.loc[170]['Lines']

'<start> if ( FSInit (  ) !=TRUE )                            <end>'